# Brain Tumor Segmentation using SLIC

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# img = cv2.imread('C:/Users/user/Downloads/final year sem 1/VIP/project/brain tumor dataset/images/181.png', cv2.IMREAD_GRAYSCALE)

# #simple global thresholding
# ret,th1 = cv2.threshold(img,90,255,cv2.THRESH_BINARY)

# plt.figure(figsize=(12,6))
# plt.subplot(121), plt.imshow(img,cmap='gray')
# plt.title('Grayscale Image')
# plt.xticks([]), plt.yticks([])
# plt.subplot(122), plt.imshow(th1,cmap='gray')
# plt.title('Binary Image (Threshold: '+str(ret)+')')
# plt.xticks([]),plt.yticks([])
# plt.show()

In [ ]:
# # morphological operations
# kernel = np.ones((5, 5), np.uint8)
# closing = cv2.morphologyEx(th1, cv2.MORPH_OPEN, kernel, iterations=7)

# # Display the original and the result
# plt.figure(figsize=(12, 6))

# plt.subplot(1, 2, 1)
# plt.title('Original Image')
# plt.imshow(th1, cmap='gray')
# plt.axis('off')

# plt.subplot(1, 2, 2)
# plt.title('Morphological Opening')
# plt.imshow(closing, cmap='gray')
# plt.axis('off')

# plt.show()

In [ ]:
# import cv2
# import numpy as np
# from skimage.segmentation import mark_boundaries
# from skimage.segmentation import slic
# import matplotlib.pyplot as plt

# # Read the image
# img = cv2.imread('C:/Users/user/Downloads/final year sem 1/VIP/project/brain tumor dataset/images/181.png', cv2.IMREAD_COLOR)

# # Convert the image from BGR to RGB (for display with matplotlib)
# img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# # Perform SLIC segmentation
# segments = slic(img, n_segments=100, compactness=10)

# # Create a mask using the segments
# mask = np.zeros_like(img)
# for seg_val in np.unique(segments):
#     mask[segments == seg_val] = np.mean(img[segments == seg_val], axis=(0, 1))

# # Display the original image and SLIC segmentation result
# fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# ax[0].imshow(img_rgb)
# ax[0].set_title('Original Image')

# ax[1].imshow(mark_boundaries(img_rgb, segments))
# ax[1].set_title('SLIC Segmentation')

# plt.show()


In [ ]:
import cv2
import numpy as np
from skimage.segmentation import slic
import matplotlib.pyplot as plt

# Function to segment brain tumor based on highest pixel intensity
def segment_tumor(img):
    # Convert the image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Find the maximum pixel intensity in the grayscale image
    max_intensity = np.max(gray_img)
    
    # Threshold the grayscale image to create a mask for the tumor region
    _, tumor_mask = cv2.threshold(gray_img, max_intensity - 10, 255, cv2.THRESH_BINARY)

    return tumor_mask

# Read the image
img = cv2.imread("C:/Users/user/Downloads/projects/Brain-Tumor-Segmentation/brain tumor dataset/preprocess/2.png", cv2.COLOR_BGR2GRAY)

# Convert the image from BGR to RGB (for display with matplotlib)
# img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_rgb = cv2.imread("C:/Users/user/Downloads/projects/Brain-Tumor-Segmentation/brain tumor dataset/preprocess/2.png", cv2.COLOR_BGR2GRAY)

# Perform SLIC segmentation
segments = slic(img, n_segments=100, compactness=10)

# Create a mask using the segments
mask = np.zeros_like(img)
for seg_val in np.unique(segments):
    mask[segments == seg_val] = np.mean(img[segments == seg_val], axis=(0, 1))

# Segment the brain tumor based on highest pixel intensity
tumor_mask = segment_tumor(mask)

# Apply the tumor mask to the original image
segmented_tumor = cv2.bitwise_and(img_rgb, img_rgb, mask=tumor_mask)

# Display the original image, SLIC segmentation result, and segmented tumor region
fig, ax = plt.subplots(1, 3, figsize=(20, 10))

ax[0].imshow(img_rgb)
ax[0].set_title('Original Image')

ax[1].imshow(mask)
ax[1].set_title('SLIC Segmentation')

ax[2].imshow(segmented_tumor)
ax[2].set_title('Segmented Tumor Region')
plt.show()

AttributeError: 'NoneType' object has no attribute 'ndim'